In [ ]:
!pip install praw
!pip install psaw
!pip install requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import praw
import pandas as pd
import datetime
import time
from difflib import SequenceMatcher

In [ ]:
#Usefull functions
def get_date(submission):
	time = submission.created
	return datetime.datetime.fromtimestamp(time).strftime("%d/%m/%Y")

#Check similatiries between two strings
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
#My credentials
reddit = praw.Reddit(client_id ='1zriW2v1Y6SJqC8gOa5bZg',
                     client_secret ='3dHrMCJ75qcXUJav18N9OG3HXt2u1w',
                     user_agent ='garra')
#                     check_for_async=False)
# to verify whether the instance is read-only instance or not
print(reddit.read_only)

True


In [ ]:
#Select the subreddit
subreddit = reddit.subreddit('facts')
# display the subreddit name
print(subreddit.display_name)
 
# display the subreddit title
# print(subreddit.title)      
 
# display the subreddit description
#print(subreddit.description)

facts


In [ ]:
#RESET THE DATAGRAME; BE CAREFULL!!!
#today = date.today()
#d1 = today.strftime("%d/%m/%Y")
#data = {"Date" : [d1], "RedditFact": ["The opposite of Microsoft Office is Macrohard Onfire"]}
#post = pd.DataFrame(data)
#post.to_excel("Reddit.xlsx")

In [ ]:
#Read the data
post = pd.read_excel("/content/drive/MyDrive/Trabajo Fin de Grado/TFG/Wrappers/Reddit.xlsx", index_col=0)

In [ ]:

#Take the posts and insert it

for submission in subreddit.top("month", limit=50):
    escape = False
    for fact in post.RedditFact:
        if similar(fact, submission.title) > 0.7:
            escape = True
    if escape == False:
        new_row = {"Date": get_date(submission), "RedditFact": submission.title}
        post = post.append(new_row, ignore_index=True)
        # print(submission.title)

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [ ]:
#Take the posts and insert it
#for submission in subreddit.top("all", limit=50):
#    if post["RedditFact"].str.contains(submission.title).any() == False:
#        new_row = {"Date": get_date(submission), "RedditFact": submission.title}
#        post = post.append(new_row, ignore_index=True)
        # print(submission.title)

In [ ]:
####Post de los últimos 2 meses, se puede ajustar para el periodo de tiempo querido, limite de 1000 post
current_timestamp = time.time()
#### 60 seconds * 60 minutes * 24 hours * 60 days = 2 months
two_months_timestamp = current_timestamp - (60 * 60 * 24 * 60)
query = 'timestamp:{}..{}'.format(current_timestamp, two_months_timestamp)
for i in reddit.subreddit('facts').search(query = query, sort='top'):
  print(i.title)

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [ ]:
#SAVE
post.to_excel("Reddit.xlsx")

In [ ]:
from psaw import PushshiftAPI
import datetime as dt
#YEAR - MONTH - DAY
start_epoch=int(dt.datetime(2017, 12, 1).timestamp())
print(start_epoch)
end_epoch=int(dt.datetime(2018, 1, 10).timestamp())
api = PushshiftAPI()
listexam = list(api.search_submissions(after=start_epoch,
                            before=end_epoch,
                            subreddit='facts',
                            sort_type="score",
                            sort="desc",
                            filter=['title'],
                            limit=100))
print(listexam)
for i in listexam:
  print(i.title)

1512086400
[submission(created_utc=1513225777, title='Aji Pai is a stupid bitch', created=1513225777.0, d_={'created_utc': 1513225777, 'title': 'Aji Pai is a stupid bitch', 'created': 1513225777.0}), submission(created_utc=1513279331, title='(Charlie Heat Version)', created=1513279331.0, d_={'created_utc': 1513279331, 'title': '(Charlie Heat Version)', 'created': 1513279331.0}), submission(created_utc=1515168944, title='Homosexuality was still classified as an illness in Sweden in 1979. Swedes responded by calling into work “sick,” saying they “felt gay.”', created=1515168944.0, d_={'created_utc': 1515168944, 'title': 'Homosexuality was still classified as an illness in Sweden in 1979. Swedes responded by calling into work “sick,” saying they “felt gay.”', 'created': 1515168944.0}), submission(created_utc=1514029632, title="22 mind-blowing science facts we didn't know at the start of 2017", created=1514029632.0, d_={'created_utc': 1514029632, 'title': "22 mind-blowing science facts we 

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
